In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

# Quiz 1: Cross-boundary anchors
    
![alt text](bbox_clip.jpg "Title")

### complete clip operations so that 'clib_boxes_graph' function operates as follows
<br>
#clipped = clip_boxes_graph(boxes, window) <br>
#sess.run(clipped) <br>
array([[  0,   0, 300, 300], <br>
      [250, 250, 450, 450], <br>
      [500, 600, 599, 700]], dtype=int32)

In [2]:
def clip_boxes_graph(boxes, window): 
    """ 
    boxes: [N, 4] where N boxes are specified in the form (y1, x1, y2, x2) 
    window: [4] in the form y1, x1, y2, x2 
    """

    # Split
    wy1, wx1, wy2, wx2 = tf.split(window, 4)
    y1, x1, y2, x2 = tf.split(boxes, 4, axis=1)
    # implement clip operation using min / max operation #
    # tf.clip_by_value 사용
    # if -> tf.cond 사용
    
    y1 = tf.maximum(y1, wy1)
    x1 = tf.maximum(x1, wx1)
    
    y2 = tf.minimum(y2, wy2)
    x2 = tf.minimum(x2, wx2)

    clipped = tf.concat([y1, x1, y2, x2], axis=1, name="clipped_boxes")
    clipped.set_shape((clipped.shape[0], 4))
    return clipped

In [3]:
window = tf.constant([0,0,599,799])
boxes = tf.constant([[-10, -10, 300, 300],
                     [250, 250, 450, 450],
                     [500, 600, 800, 700]])

In [4]:
clipped = clip_boxes_graph(boxes, window)

In [5]:
sess.run(clipped)

array([[  0,   0, 300, 300],
       [250, 250, 450, 450],
       [500, 600, 599, 700]])

---

# Quiz 2: NMS(non-maximum suppression)

![alt text](nms.jpg "Title")

### insert values of "scores" tensor so that nms function outputs <br>

#sess.run(proposals) <br>
array([[  1.,   1., 298., 299.]], dtype=float32)

In [6]:
# hint: https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression?hl=en
def nms(boxes, scores):
    indices = tf.image.non_max_suppression(
        boxes, scores, 1, name="rpn_nms")
    proposals = tf.gather(boxes, indices)
    
    # Pad if needed
    padding = tf.maximum(1-tf.shape(proposals)[0], 0)
    proposals = tf.pad(proposals, [(0, padding), (0, 0)])
    return proposals

In [7]:
boxes = tf.constant([[0., 0., 300., 300.],
                     [0., 0., 301., 299.],
                     [1., 1., 298., 299.]])
scores = tf.constant([0., 0., 1.])

In [8]:
proposals = nms(boxes, scores)

In [9]:
sess.run(proposals)

array([[  1.,   1., 298., 299.]], dtype=float32)

---

# Quiz 3: Smooth L1 Loss

![alt text](smooth_l1.JPG "Title")

### implement Smooth L1 Loss

In [72]:
def smooth_l1_loss(prediction, target):
    prediction = tf.to_float(prediction)
    target = tf.to_float(target)
    
    # implement smooth_l1_loss calculating code below
    ads_diff = tf.abs(tf.subtract(prediction, target))
    def result1():
        return tf.multiply(0.5, tf.square(ads_diff))
    def result2():
        return tf.subtract(ads_diff,0.5)
    loss = tf.cond(tf.less(ads_diff,1.0), result1, result2)
  
    return loss

In [73]:
prediction1 = tf.constant(3)
target1 = tf.constant(5)
loss1 = smooth_l1_loss(prediction1, target1)

prediction2 = tf.constant(3)
target2 = tf.constant(3.1)
loss2 = smooth_l1_loss(prediction2, target2)

In [74]:
sess.run([loss1, loss2])

[1.5, 0.0049999906]

----

# Quiz 4: Binary Cross-Entropy Loss

![alt text](bce.jpg "Title")

In [78]:
def binary_cross_entropy_loss(prediction, target, eps=1e-9):
    '''
    prediction : (N, 2)-shaped, float tensor
    target : (N)-shaped, float tensor
    eps : very small number for numerical stability
    '''
    # implement BCE here
    # log 내 eps 값을 더해준다.
    loss = -1 * target * tf.log(prediction + eps) - (1 - target) * tf.log((1 - prediction) + eps)
    return loss

In [79]:
prediction1 = tf.constant(1.)
target1 = tf.constant(1.)
loss1 = binary_cross_entropy_loss(prediction1, target1)

prediction2 = tf.constant(0.)
target2 = tf.constant(1.)
loss2 = binary_cross_entropy_loss(prediction2, target2)

prediction3 = tf.constant(0.8)
target3 = tf.constant(1.0)
loss3 = binary_cross_entropy_loss(prediction3, target3)

In [80]:
sess.run([loss1, loss2, loss3])

[0.0, 20.723267, 0.22314353]

---

# Quiz 5: IoU (Intersection-over-Union)

![alt text](iou.jpg "Title")

In [84]:
def iou(box1, box2):
    '''
    box1 : (4)-shaped float tensor. (b1_y1, b1_x1, b1_y2, b1_x2)
    box2 : (4)-shaped float tensor. (b2_y1, b2_x1, b2_y2, b2_x2)
    '''
    
    b1_y1, b1_x1, b1_y2, b1_x2 = tf.split(box1, 4)
    b2_y1, b2_x1, b2_y2, b2_x2 = tf.split(box2, 4)
    
    # implement iou here  
    intersection = tf.abs(b1_x2 - b2_x1) * tf.abs(b1_y2 - b2_y1)
    box1_area = tf.abs(b1_x2 - b1_x1) * tf.abs(b1_y2 - b1_y1)
    box2_area = tf.abs(b2_x2 - b2_x1) * tf.abs(b2_y2 - b2_y1)
    union = box1_area + box2_area - intersection
    
    score = intersection/union
    return score

In [85]:
box1 = tf.constant([0.,0.,300.,300.])
box2 = tf.constant([100.,100.,400.,400.])

In [86]:
overlap_score = iou(box1, box2)

In [87]:
sess.run(overlap_score)

array([0.2857143], dtype=float32)

# Quiz 6 : Bbox refinement via regression

![alt text](bbox_regression.jpg "Title")

In [92]:
def refine_bbox(anchor_box, regression):
    '''
    anchor_box : (4)-shaped anchor bbox coordinates (y1, x1, y2, x2) 초록색 박스
    regression : (4)-shaped regressed parameter for bbox refinement (dy, dx, sy, sx) 붉은색 박스 - 위치평행이동, 크기늘리기
    '''
    y1, x1, y2, x2 = tf.split(anchor_box, 4)
    dy, dx, sy, sx = tf.split(regression, 4)
    
    # implement refine code here
    h = tf.abs(y2 - y1) * sy
    w = tf.abs(x2 - x1) * sx
    
    new_y1 = y1 + dy
    new_x1 = x1 + dx
    new_y2 = new_y1 + h
    new_x2 = new_x1 + w
    
    refined_box = tf.concat([new_y1, new_x1, new_y2, new_x2], axis=0)
    return refined_box    

In [93]:
anchor_box = tf.constant([0.,0.,300.,300.])
regression = tf.constant([5, 5, 0.5, 0.5])

In [94]:
refined_box = refine_bbox(anchor_box, regression)

In [95]:
sess.run(refined_box)

array([  5.,   5., 155., 155.], dtype=float32)